In [230]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import re

In [231]:
df_origin_test= pd.read_csv('data/test_set.csv', index_col=0)
df_origin_train = pd.read_csv('data/train_set.csv', index_col=0)

df_train = df_origin_train.copy()
df_test = df_origin_test.copy()

In [232]:
df = pd.concat([df_train, df_test], ignore_index=True)
df.shape

(100000, 27)

Delete unuseful columns

In [233]:
#这里不能删除ID因为在submission时要以他作为索引
del df['Name'] # Name of client 
del df['SSN'] # SSN (social security number of a person)

Normalization

In [234]:
df.describe()

,Customer_ID,Month,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,25982.666640,4.500000,33.316340,50505.123449,4197.270835,5.368820,5.533570,14.53208,3.532880,21.08141,13.313120,10.470323,5.798250,1426.220376,32.285173,221.220460,107.699208,55.101315,392.697586
std,14340.543051,2.291299,10.764812,38299.422093,3186.432497,2.593314,2.067098,8.74133,2.446356,14.80456,6.237166,6.609481,3.867826,1155.129026,5.116875,99.680716,132.267056,39.006932,201.652719
min,1006.000000,1.000000,14.000000,7005.930000,303.645417,0.000000,0.000000,1.00000,0.000000,0.00000,0.000000,0.500000,0.000000,0.230000,20.000000,1.000000,0.000000,0.000000,0.007760
25%,13664.500000,2.750000,24.000000,19342.972500,1626.594167,3.000000,4.000000,7.00000,2.000000,10.00000,9.000000,5.380000,3.000000,566.072500,28.052567,144.000000,29.268886,27.959111,267.615983
50%,25777.000000,4.500000,33.000000,36999.705000,3095.905000,5.000000,5.000000,13.00000,3.000000,18.00000,14.000000,9.400000,5.000000,1166.155000,32.305784,219.000000,66.462304,45.156550,333.865366
75%,38385.000000,6.250000,42.000000,71683.470000,5957.715000,7.000000,7.000000,20.00000,5.000000,28.00000,18.000000,14.850000,8.000000,1945.962500,36.496663,302.000000,147.392573,71.295797,463.215683
max,50999.000000,8.000000,56.000000,179987.280000,15204.633333,11.000000,11.000000,34.00000,9.000000,62.00000,25.000000,29.980000,17.000000,4998.070000,50.000000,404.000000,1779.103254,434.191089,1183.930696


In [235]:
scaler = MinMaxScaler()
col_float = ['Age','Annual_Income','Delay_from_due_date','Num_of_Delayed_Payment',
             'Outstanding_Debt','Credit_History_Age','Total_EMI_per_month','Monthly_Balance']
for i in df[col_float]:
  df[i] = scaler.fit_transform(df[[i]])
df.head()

"scaler = MinMaxScaler()\ncol_float = ['Age','Annual_Income','Delay_from_due_date','Num_of_Delayed_Payment',\n             'Outstanding_Debt','Credit_History_Age','Total_EMI_per_month','Monthly_Balance']\nfor i in df[col_float]:\n  df[i] = scaler.fit_transform(df[[i]])\ndf.head()"

In [236]:
#非数值列名
object_col = df.describe(include='O').columns
object_col

Index(['Occupation', 'Type_of_Loan', 'Credit_Mix', 'Payment_of_Min_Amount',
       'Payment_Behaviour', 'Credit_Score'],
      dtype='object')

In [237]:
#Payment_Behaviour
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('Low_spent_Small_value_payments','1')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('Low_spent_Medium_value_payments','2')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('Low_spent_Large_value_payments','3')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('High_spent_Small_value_payments','4')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('High_spent_Medium_value_payments','5')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('High_spent_Large_value_payments','6')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(float)

In [238]:
#非数值列名
object_col = df.describe(include='O').columns
object_col

Index(['Occupation', 'Type_of_Loan', 'Credit_Mix', 'Payment_of_Min_Amount',
       'Credit_Score'],
      dtype='object')

In [239]:
#Credit_Score & Payment_of_Min_Amount
df['Credit_Score'] = df['Credit_Score'].str.replace('Good', '3', n=-1)
df['Credit_Score'] = df['Credit_Score'].str.replace('Standard', '2', n=-1)
df['Credit_Score'] = df['Credit_Score'].str.replace('Poor', '1', n=-1)
df['Credit_Score'] = df[['Credit_Score']].apply(pd.to_numeric)

df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].str.replace('NM', '0')
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].str.replace('Yes', '1')
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].str.replace('No', '2')
df['Payment_of_Min_Amount'] = df[['Payment_of_Min_Amount']].apply(pd.to_numeric)


In [240]:
#非数值列名
object_col = df.describe(include='O').columns
object_col

Index(['Occupation', 'Type_of_Loan', 'Credit_Mix'], dtype='object')

In [241]:
### Balancing - Occupation
occupation_count = df['Occupation'].value_counts(dropna = False)
occupation_count

Occupation
Lawyer           7096
Engineer         6864
Architect        6824
Mechanic         6776
Scientist        6744
Accountant       6744
Developer        6720
Media_Manager    6720
Teacher          6672
Entrepreneur     6648
Doctor           6568
Journalist       6536
Manager          6432
Musician         6352
Writer           6304
Name: count, dtype: int64

In [242]:
df = pd.get_dummies(df, prefix='Occupation', columns=['Occupation'])

df.head()


"df = pd.get_dummies(df, prefix='Occupation', columns=['Occupation'])\n\ndf.head()\n"

In [243]:
#非数值列名
object_col = df.describe(include='O').columns
object_col

Index(['Occupation', 'Type_of_Loan', 'Credit_Mix'], dtype='object')

In [244]:
df['Credit_Mix'].value_counts()
m = {"Bad":0, "Standard":1, "Good":2}
df['Credit_Mix'] = df['Credit_Mix'].map(m)

In [245]:
#非数值列名
object_col = df.describe(include='O').columns
object_col

Index(['Occupation', 'Type_of_Loan'], dtype='object')

In [246]:
## Rebuild Type of loans Columns 
for i in df['Type_of_Loan'].value_counts().head(9).index[1:] : 
    df[i] = df['Type_of_Loan'].str.contains(i)

del df['Type_of_Loan']

df_cleaned = df.copy()

In [247]:
df_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 32 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  int64  
 1   Month                     100000 non-null  int64  
 2   Age                       100000 non-null  float64
 3   Occupation                100000 non-null  object 
 4   Annual_Income             100000 non-null  float64
 5   Monthly_Inhand_Salary     100000 non-null  float64
 6   Num_Bank_Accounts         100000 non-null  float64
 7   Num_Credit_Card           100000 non-null  float64
 8   Interest_Rate             100000 non-null  float64
 9   Num_of_Loan               100000 non-null  float64
 10  Delay_from_due_date       100000 non-null  float64
 11  Num_of_Delayed_Payment    100000 non-null  float64
 12  Changed_Credit_Limit      100000 non-null  float64
 13  Num_Credit_Inquiries      100000 non-null  fl

In [248]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 32 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  int64  
 1   Month                     100000 non-null  int64  
 2   Age                       100000 non-null  float64
 3   Occupation                100000 non-null  object 
 4   Annual_Income             100000 non-null  float64
 5   Monthly_Inhand_Salary     100000 non-null  float64
 6   Num_Bank_Accounts         100000 non-null  float64
 7   Num_Credit_Card           100000 non-null  float64
 8   Interest_Rate             100000 non-null  float64
 9   Num_of_Loan               100000 non-null  float64
 10  Delay_from_due_date       100000 non-null  float64
 11  Num_of_Delayed_Payment    100000 non-null  float64
 12  Changed_Credit_Limit      100000 non-null  float64
 13  Num_Credit_Inquiries      100000 non-null  fl

In [249]:
df_cleaned.drop(['Month', 'Customer_ID',
 'Credit_Utilization_Ratio','Payment_Behaviour','Occupation',
 'Credit_Mix','Payment_of_Min_Amount','Amount_invested_monthly',
 'Monthly_Balance','Num_of_Loan'], axis=1,inplace=True)

In [250]:
df_cleaned.head()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,...,Total_EMI_per_month,Credit_Score,not specified,credit-builder loan,personal loan,debt consolidation loan,student loan,payday loan,mortgage loan,auto loan
0,18.0,70260.20,5564.016667,9.0,7.0,20.0,60.0,24.0,9.97,8.0,...,246.378645,1.0,True,False,False,False,True,True,True,False
1,21.0,18001.59,1258.132500,7.0,9.0,31.0,20.0,19.0,16.34,12.0,...,24.986447,1.0,False,True,False,False,False,False,True,False
2,47.0,9824.31,707.692500,7.0,4.0,17.0,24.0,17.0,0.96,5.0,...,0.000000,3.0,False,False,False,False,False,False,False,False
3,41.0,87481.62,7022.135000,0.0,4.0,3.0,14.0,6.0,1.45,3.0,...,55.653369,2.0,False,True,False,False,False,False,False,False
4,53.0,129204.92,10508.076667,5.0,6.0,12.0,8.0,19.0,1.54,1.0,...,277.610885,2.0,False,True,True,False,False,True,False,True


### Data Splitting 

In [273]:
columns_to_include = [
    'Total_EMI_per_month', 'Num_Bank_Accounts', 'Num_of_Delayed_Payment',
    'Delay_from_due_date', 'Changed_Credit_Limit', 'Num_Credit_Card',
    'Outstanding_Debt', 'Interest_Rate', 'Credit_Mix','Credit_Score'
]

df_final = df[columns_to_include]

In [274]:
df_final.head().T

,0,1,2,3,4
Total_EMI_per_month,246.378645,24.986447,0.00,55.653369,277.610885
Num_Bank_Accounts,9.000000,7.000000,7.00,0.000000,5.000000
Num_of_Delayed_Payment,24.000000,19.000000,17.00,6.000000,19.000000
Delay_from_due_date,60.000000,20.000000,24.00,14.000000,8.000000
Changed_Credit_Limit,9.970000,16.340000,0.96,1.450000,1.540000
Num_Credit_Card,7.000000,9.000000,4.00,4.000000,6.000000
Outstanding_Debt,2228.790000,2225.580000,1233.96,214.430000,1075.370000
Interest_Rate,20.000000,31.000000,17.00,3.000000,12.000000
Credit_Mix,0.000000,0.000000,1.00,2.000000,1.000000
Credit_Score,1.000000,1.000000,3.00,2.000000,2.000000


In [275]:
#把训练集和测试集分开
# Separate the training set where 'Credit_Score' is not null
training_set = df_final[df_final['Credit_Score'].notna()]

# Separate the test set where 'Credit_Score' is null
test_set = df_final[df_final['Credit_Score'].isna()]

# Feature to predict
label_to_predict = 'Credit_Score'

In [276]:
# define dataset
X, y = training_set.drop("Credit_Score",axis=1).values , training_set["Credit_Score"] 

In [277]:
#数据不平衡，因此不能使用过采样的方法来提高平衡
y.value_counts(normalize=True)

Credit_Score
2.0    0.529714
1.0    0.291414
3.0    0.178871
Name: proportion, dtype: float64

In [278]:
from imblearn.over_sampling import SMOTE
rus = SMOTE(sampling_strategy='auto')
X_data_rus, y_data_rus = rus.fit_resample(X, y)

In [279]:
#通过 SMOTE 方法增加了数据中少数类别的样本数，改善了类别间的不平衡
#有助于提高模型对少数类别的预测能力。
y_data_rus.value_counts(normalize=True)

Credit_Score
1.0    0.333333
3.0    0.333333
2.0    0.333333
Name: proportion, dtype: float64

In [280]:
# 将浮点数标签转换为整数标签
y_data_rus_int = y_data_rus.astype(int)


In [281]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_data_rus, y_data_rus_int, test_size=0.3, random_state=42,stratify=y_data_rus)

In [282]:
from sklearn.preprocessing import PowerTransformer
#Using Power transformer to avoid Data Skewness
scalar = PowerTransformer(method='yeo-johnson', standardize=True).fit(X_train)

In [283]:
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)

### Modeling and evaluation

In [284]:
# Modeling and evaluation 
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import (
    BaggingClassifier,
    ExtraTreesClassifier,
    RandomForestClassifier,
    StackingClassifier,
    HistGradientBoostingClassifier
)
from xgboost import XGBClassifier
from sklearn.metrics import classification_report 
import joblib

In [285]:
bagging = BaggingClassifier(n_jobs=-1)
extraTrees = ExtraTreesClassifier(max_depth=10, n_jobs=-1)
randomForest = RandomForestClassifier(n_jobs=-1)
histGradientBoosting = HistGradientBoostingClassifier()
XGB = XGBClassifier(n_jobs=-1)

model = StackingClassifier([
    ('bagging', bagging),
    ('extraTress', extraTrees),
    ('randomforest', randomForest),
    ('histGradientBoosting', histGradientBoosting),
    ('XGB', XGB)
], n_jobs=-1)

In [286]:
#model fitting
model.fit(X_train, y_train)

/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('bagging', BaggingClassifier(n_jobs=-1)),
                               ('extraTress',
                                ExtraTreesClassifier(max_depth=10, n_jobs=-1)),
                               ('randomforest',
                                RandomForestClassifier(n_jobs=-1)),
                               ('histGradientBoosting',
                                HistGradientBoostingClassifier()),
                               ('XGB',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None...
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=-1,
                                              num_parallel_tree=None,
                                              random_state=None, ...))],
                   n_jobs=-1)

### Model Evaluation 

In [287]:
print("Train Score: ",model.score(X_train, y_train))

Train Score:  0.9529845379359942


In [288]:
print("Test Score: ",model.score(X_test, y_test))

Test Score:  0.8611410763514323


In [289]:
y_pred = model.predict(X_test)

In [290]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           1       0.88      0.86      0.87     11385
           2       0.78      0.82      0.80     10502
           3       0.92      0.89      0.91     11485

    accuracy                           0.86     33372
   macro avg       0.86      0.86      0.86     33372
weighted avg       0.86      0.86      0.86     33372



### Generating the submission

In [291]:
#去掉目标column
X_test_set = test_set.drop("Credit_Score", axis=1)

In [292]:
# 使用模型进行预测
predictions = model.predict(X_test_set)

/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but BaggingClassifier was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/Users/zhengwan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [293]:

X_test_set['Predicted_Credit_Score'] = predictions

a= X_test_set['Predicted_Credit_Score'].value_counts()
a

Predicted_Credit_Score
1    29433
2      566
3        1
Name: count, dtype: int64